Рассмотрим основные моменты не вошедшие в предыдущие лекции

Обновим и подключим библиотеки:

In [0]:
!pip install --upgrade pip
!pip install --upgrade scikit-learn==0.23.0


     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 7.3 MB 2.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

Импортируем датасет MNIST.

In [0]:
from keras.datasets import mnist
(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 9s 1us/step


In [0]:
dim = 784 # 28*28 
X_train = X_train.reshape(len(X_train), dim)

Введите количество главных компонент:

In [0]:
pca = PCA(n_components=41, svd_solver='full')
modelPCA = pca.fit(X_train) 
X_train = modelPCA.transform(X_train)

In [0]:
explained_variance = np.round(np.cumsum(pca.explained_variance_ratio_),3)
explained_variance

array([0.097, 0.168, 0.23 , 0.284, 0.332, 0.375, 0.408, 0.437, 0.465,
       0.488, 0.509, 0.529, 0.547, 0.564, 0.579, 0.594, 0.607, 0.62 ,
       0.632, 0.644, 0.654, 0.664, 0.674, 0.683, 0.692, 0.7  , 0.708,
       0.716, 0.724, 0.731, 0.737, 0.744, 0.75 , 0.755, 0.761, 0.767,
       0.772, 0.777, 0.781, 0.786, 0.791])

Для разделения данных рекомендуем использовать функцию train_test_split из библиотеки sklearn. Следующий код разделяет исходные данные в отношении 70/30.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=28)

Выборочное среднее нулевой колонки для полученного тренировочного набора:

In [0]:
n = len(X_train)
X = 0
for arr in X_train:
  X += arr[0]
print(X/n)

4.332942081334197


Примените полученное ранее преобразование метода главных компонент к новым данным


In [0]:
test = pd.read_csv('/pred_for_task.csv', sep=',')
y_true = np.array(test['Label'])

# New Section

In [0]:
test.drop(columns = ['FileName', 'Label'], inplace=True)
X = np.array(test)
X = modelPCA.transform(X)

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


Обучите алгоритм случайного леса RandomForestClassifier()

In [0]:
tree = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=28)
clf = OneVsRestClassifier(tree).fit(X_train, y_train)

In [0]:
y_pred = clf.predict(X_test)

Количество верно классифицированных объектов класса 3 для тестовых данных

In [0]:
CM = confusion_matrix(y_test, y_pred)
CM[5][5]

1478

Вероятность отнесения изображения file18 к назначенному классу алгоритмом случайного леса

In [0]:
y_pred = clf.predict_proba([X[0]])
label =  y_true[0]
print(label, '--prob', np.round(y_pred[0][label], 4))

7 --prob 0.956


Обучите алгоритм логистической регрессии LogisticRegression

In [0]:
lr = LogisticRegression(solver='lbfgs', random_state=28, max_iter=7600)
clf = OneVsRestClassifier(lr).fit(X_train, y_train)

In [0]:
y_pred = clf.predict(X_test)
CM = confusion_matrix(y_test, y_pred)

In [0]:
CM[5][5]

1332

Вероятность отнесения изображения file15 к назначенному классу алгоритмом логистической регрессией.

In [0]:
y_pred = clf.predict_proba([X[28]])
label =  y_true[28]
print(label, '--prob', np.round(y_pred[0][label], 4))

0 --prob 0.981


Обучите дерево принятия решений DecisionTreeClassifier()

In [0]:
decTree = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=28)
clf = OneVsRestClassifier(decTree).fit(X_train, y_train)

In [0]:
y_pred = clf.predict(X_test)
CM = confusion_matrix(y_test, y_pred)

In [0]:
CM[7][7]

1682

Введите вероятность отнесения изображения file19 к назначенному классу деревом принятия решений.

In [0]:
y_pred = clf.predict_proba([X[3]])
label =  y_true[3]
print(label, '--prob', np.round(y_pred[0][label], 4))

0 --prob 0.9947
